# Analyze simulation
adopted from [buffet_oat15](https://github.com/JanisGeise/buffet_oat15/tree/jgeise) to quickly post-process and assess a single steady-state simulation

In [ ]:
import torch as pt
import sys
from glob import glob
from os import environ, system
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

from os import makedirs
import pandas as pd
from os.path import join, exists
from flowtorch.data import CSVDataloader

environ["airfoil_shape_optimization"] = ".."
sys.path.insert(0, environ["airfoil_shape_optimization"])

from airfoil_shape_optimization.generate_airfoil import AirfoilGenerator
from utils import load_yplus, load_force_coeffs, compute_camber_line, load_residuals, get_loss_from_log_file

In [ ]:
# flow quantities
u_inf = 20
p_inf = 0
rho_inf = 1

# chord length
chord = 0.15

# use latex fonts
plt.rcParams.update({"text.usetex": True, "figure.dpi": 360, "text.latex.preamble": r"\usepackage{xcolor}"})

# use default color cycle
color = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f"]

# use these linestyles
ls = ["-", "--", "-.", ":"]

In [ ]:
# define load and save path
load_dir = join("..", "run", "test_polar_run", "first_test")

save_dir = join("..", "run", "test_polar_run", "first_test", "plots")
cases = ["validation_run"]
legend = [r"airfoil polar test $\#1$"]

# create plot directory
if not exists(save_dir):
    makedirs(save_dir)

In [ ]:
# load residuals TODO: rename solverInfo etc. files as well for validation run
residuals = [load_residuals(join(load_dir, c)) for c in cases]

# plot residuals
for c in range(len(cases)):
    fig, ax = plt.subplots(2, 1, figsize=(6, 5), sharex="col")
    for k in residuals[c].keys():
        if k.endswith("_initial") and not k == "t":
            ax[0].plot(residuals[c]["t"], residuals[c][k], label=k)
    
        elif k.endswith("_final") and not k == "t":
            ax[1].plot(residuals[c]["t"], residuals[c][k], label=k)
    
    for i in range(2):
        # ax[i].axvline(750, color="black", ls="--")
    
        ax[i].set_yscale("log")
        ax[i].set_xscale("log")
        ax[i].set_xlim(residuals[c]["t"].min(), residuals[c]["t"].max())
        ax[i].legend(ncols=2, loc="lower left", facecolor="white", framealpha=1)
    
        ax[i].grid(visible=True, which="major", linestyle="-", alpha=0.45, color="black", axis="y")
        ax[i].minorticks_on()
        ax[i].tick_params(axis="x", which="minor", bottom=False)
        ax[i].grid(visible=True, which="minor", linestyle="--", alpha=0.35, color="black", axis="y")
    
    ax[-1].set_xlabel(r"iteration no. $\#$")
    fig.tight_layout()
    plt.savefig(join(save_dir, f"residuals_vs_iterations_case{c}.png"))

In [ ]:
# check forces 
forces = [load_force_coeffs(join(load_dir, c), "alpha_0.000000") for c in cases]

ls = ["-"] * len(forces)

# use default color cycle
color = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f"]

fig, ax = plt.subplots(3, 1, figsize=(6, 4), sharex="col")
for i in range(len(cases)):
    ax[0].plot(forces[i].t, forces[i].cx, label=legend[i], ls=ls[i], color=color[i])
    ax[1].plot(forces[i].t, forces[i].cy, ls=ls[i], color=color[i])
    ax[2].plot(forces[i].t, forces[i].cm_pitch, ls=ls[i], color=color[i])

ax[0].set_ylim(0.0002, 0.01)
ax[1].set_ylim(-0.2, 0.65)
ax[2].set_ylim(0.1, 0.5)
ax[0].set_ylabel(r"$c_d$")
ax[1].set_ylabel(r"$c_l$")
ax[2].set_ylabel(r"$c_{m, pitch}$")
ax[-1].set_xlabel(r"iteration no. $\#$")
ax[-1].set_xlim(0, forces[0].t.max())

fig.tight_layout()
for i in range(3):
    ax[i].grid(visible=True, which="major", linestyle="-", alpha=0.45, color="black", axis="y")
    ax[i].minorticks_on()
    ax[i].tick_params(axis="x", which="minor", bottom=False)
    ax[i].grid(visible=True, which="minor", linestyle="--", alpha=0.35, color="black", axis="y")

fig.legend(ncol=3, loc="upper center")
fig.subplots_adjust(top=0.9)
plt.savefig(join(save_dir, "comparison_coefficients_vs_t.png"))

In [ ]:
# plot max y+ vs. time
y_plus = [load_yplus(join(load_dir, c)) for c in cases]

fig, ax = plt.subplots(1, 1, figsize=(6, 3))
for i in range(len(cases)):
    ax.plot(y_plus[i].t, y_plus[i].yPlus_max, label=legend[i], ls=ls[i])
        
ax.set_xlabel(r"iteration no. $\#$")
ax.set_xlim(0, forces[0].t.max())
ax.set_ylabel("$y^+_{max}$")
ax.set_ylim(0, 3)
fig.tight_layout()
ax.grid(visible=True, which="major", linestyle="-", alpha=0.45, color="black", axis="y")
ax.minorticks_on()
ax.tick_params(axis="x", which="minor", bottom=False)
ax.grid(visible=True, which="minor", linestyle="--", alpha=0.35, color="black", axis="y")
fig.legend(ncol=3, loc="upper center")
fig.subplots_adjust(top=0.86)
plt.savefig(join(save_dir, "comparison_yPlus_max_vs_t.png"))

In [ ]:
# plot avg. y+ vs. time
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
for i in range(len(cases)):
    ax.plot(y_plus[i].t, y_plus[i].yPlus_avg, label=legend[i], ls=ls[i])
        
ax.set_xlabel(r"iteration no. $\#$")
ax.set_xlim(0, forces[0].t.max())
ax.set_ylabel("$y^+_{avg}$")
ax.set_ylim(0.5, 1.5)
fig.tight_layout()
ax.grid(visible=True, which="major", linestyle="-", alpha=0.45, color="black", axis="y")
ax.minorticks_on()
ax.tick_params(axis="x", which="minor", bottom=False)
ax.grid(visible=True, which="minor", linestyle="--", alpha=0.35, color="black", axis="y")
fig.legend(ncol=3, loc="upper center")
fig.subplots_adjust(top=0.88)
plt.savefig(join(save_dir, "comparison_yPlus_avg_vs_t.png"))

In [ ]:
x, z, cp, camber = [], [], [], []

# TODO: mv to fct
airfoil_generator = AirfoilGenerator(x_stop=0.15)

for i, c in enumerate(cases):
    # TODO: refactor so mean cp or other quantitiy can be computed as well as instantaneous quantities
    loader = CSVDataloader.from_foam_surface(join(load_dir, c, "postProcessing", "surface"),
                                                 f"p_airfoil.raw")

    x_temp = loader.vertices[:, 0]
    z_temp = loader.vertices[:, 2]

    # TODO: mv to fct
    final_params = pt.load(join(load_dir, "results_final_parameters.pt"), weights_only=False)[0]
    
    x_af, y_af = airfoil_generator.generate_airfoil(final_params["N1"], final_params["N2"], final_params["KR"],
                                                    final_params["f_max"], final_params["xf"],
                                                    final_params["t_max"], write_file=False)
    x_ss, y_ss = x_af[:x_af.size(0)//2+1], y_af[:x_af.size(0)//2+1]
    x_ps, y_ps = x_af[x_af.size(0)//2:], y_af[x_af.size(0)//2:]
    _y_middle = y_ps + (reversed(y_ss) - y_ps) / 2

    # interpolate onto actual x-coordinates
    camber_interp = interp1d(x_ps, _y_middle, kind='linear', fill_value='extrapolate')
    camber_line = pt.tensor(camber_interp(x_temp.numpy()))

    # get all coordinates for suction and pressure side
    is_suction = z_temp > camber_line
    is_pressure = ~is_suction
    
    cp_temp = loader.load_snapshot("p", loader.write_times[-1])
    cp_suction = cp_temp[is_suction]
    cp_pressure = cp_temp[is_pressure]

    x_suction = x_temp[is_suction]
    z_suction = z_temp[is_suction]
    x_pressure = x_temp[is_pressure]
    z_pressure = z_temp[is_pressure]

    idx_suction = pt.argsort(x_suction, descending=True)
    x_suction = x_suction[idx_suction]
    z_suction = z_suction[idx_suction]
    cp_suction = cp_suction[idx_suction]

    idx_pressure = pt.argsort(x_pressure)
    x_pressure = x_pressure[idx_pressure]
    z_pressure = z_pressure[idx_pressure]
    cp_pressure = cp_pressure[idx_pressure]

    x_sorted = pt.cat([x_suction, x_pressure])
    z_sorted = pt.cat([z_suction, z_pressure])
    cp_sorted = pt.cat([cp_suction, cp_pressure])

    x.append(x_sorted)
    z.append(z_sorted)
    cp.append((cp_sorted - p_inf) / (0.5 * rho_inf * u_inf**2))
    # we interpolated the camber line at the same positions as x_ps
    camber.append([x_temp, camber_line])

In [ ]:
# test plot airfoil with computed camber line
for c in range(len(cases)):
    fig, ax = plt.subplots(figsize=(6, 2))
    ax.plot(x[c] / chord, z[c] / chord, color=color[0])
    ax.scatter(camber[c][0] / chord, camber[c][1] / chord, color=color[1], marker=".", s=2)
    ax.set_aspect("equal")
    ax.set_xlabel("$x / c$")
    ax.set_ylabel("$z / c$")
    fig.tight_layout()
    plt.savefig(join(save_dir, f"airfoil_case_{c}.png"))

In [ ]:
# plot mean cp-distribution
fig, ax = plt.subplots(figsize=(6, 4))
for j in range(len(cases)):
    ax.plot(x[j]/chord, cp[j], zorder=10, color=color[j], label=f"{legend[j]}")
ax.set_xlabel(r"$x/c$")
ax.set_ylabel("$c_p$")
ax.grid(visible=True, which="major", linestyle="-", alpha=0.35, color="black", axis="both")
ax.minorticks_on()
ax.tick_params(axis="x", which="minor", bottom=False)
ax.grid(visible=True, which="minor", linestyle="--", alpha=0.25, color="black", axis="both")
plt.gca().invert_yaxis()
fig.tight_layout()
fig.legend(ncol=3, loc="upper center")
fig.subplots_adjust(top=0.9)
plt.savefig(join(save_dir, f"comparison_cp_distribution_alpha3.5deg.png"))

In [ ]:
# plot alpha-cl and cd-cl polars during the optimization TODO: mv to function
polar_data = [pd.read_csv(p, sep=r"\s+", comment="#", header=None, usecols=[0, 1, 2, 3], names=["alpha", "cd", "cl", "cm_pitch"], 
                          skiprows=2) for p in sorted(glob(join(load_dir, f"polar_trial_*.dat")), 
                                                      key=lambda x: int(x.split("_")[-1].split(".")[0]))]

fig, ax = plt.subplots(1, 2, figsize=(6, 3), sharey="row")
for i, polar in enumerate(polar_data):
    ax[0].plot(polar["alpha"], polar["cl"], color="black", alpha=(i+1)/(len(polar_data)+1))
    ax[1].plot(polar["cd"], polar["cl"], color="black", alpha=(i+1)/(len(polar_data)+1))
ax[0].set_xlabel(r"$\alpha$")
ax[1].set_xlabel(r"$c_d$")
ax[0].set_ylabel(r"$c_l$")
ax[0].set_xlim(polar["alpha"].min(), polar["alpha"].max())
fig.tight_layout()
plt.savefig(join(save_dir, "polar_vs_trial"))
plt.show()

In [ ]:
# plot alpha-cl and cd-cl polars for the final airfoil
polar_data = [pd.read_csv(join(load_dir, c, "polar_trial_0.dat"), sep=r"\s+", comment="#", header=None, usecols=[0, 1, 2, 3], 
                         names=["alpha", "cd", "cl", "cm_pitch"], skiprows=2) for c in cases]

fig, ax = plt.subplots(1, 2, figsize=(6, 3), sharey="row")
for i, polar in enumerate(polar_data):
    ax[0].plot(polar["alpha"], polar["cl"], label=legend[i])
    ax[1].plot(polar["cd"], polar["cl"])
ax[0].set_xlabel(r"$\alpha$")
ax[1].set_xlabel(r"$c_d$")
ax[0].set_ylabel(r"$c_l$")
ax[0].set_xlim(polar["alpha"].min(), polar["alpha"].max())
fig.tight_layout()
fig.legend(ncol=3, loc="upper center")
fig.subplots_adjust(top=0.88)
plt.savefig(join(save_dir, "polar_final_airfoils"))
plt.show()

In [ ]:
# load the loss vs. trials
loss = [get_loss_from_log_file(join(load_dir, c)) for c in cases]

# plot the loss vs. trial
fig, ax = plt.subplots(figsize=(6, 3))
for i, l in enumerate(loss):
    ax.plot(l, marker="x", label=legend[i])
ax.set_xlabel(r"trial no. $\#$")
ax.set_xlim(0, len(loss[0])-1)
ax.set_ylabel("objective")
ax.grid(visible=True, which="major", linestyle="-", alpha=0.35, color="black", axis="both")
ax.minorticks_on()
ax.tick_params(axis="x", which="minor", bottom=False)
ax.grid(visible=True, which="minor", linestyle="--", alpha=0.25, color="black", axis="both")
fig.tight_layout()
plt.savefig(join(save_dir, "trial_vs_objective"))